[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/trevortknguyen/nonparametric/blob/master/product_kernels.ipynb)

# Understanding product kernels
A product kernel is a way to do multivariate kernel density estimation where instead of using a covariance matrix as a bandwidth, you take a product of multiple univariate kernels, one for each dimension.

The product kernel seems similar to just using a diagonal covariance matrix, but they are not equivalent because the diagonal bandwidth matrix implies the dimensions are all independent. The product kernel does not assume independent dimensions/features.

If it did, you could take the product of all of the dimensions/features at the end versus the beginning. This document explains this a bit. http://www.complexity.co.kr/wp-content/uploads/2015/02/pr_l7.pdf

In [ ]:
import scipy.stats
import numpy as np
import seaborn as sns
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt

import math

## Step 1: Check out ways to simulate multivariate distributions
This code comes from the `scipy` documentation. It allows specifying a full covariance matrix for the spread of the Gaussian. We probably don't need this and can specify the identity matrix.

In [ ]:
x, y = np.mgrid[-1:1:.01, -1:1:.01]
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y
rv = scipy.stats.multivariate_normal([0.5, -0.2], [[2.0, 0.3], [0.3, 0.5]])
plt.contourf(x, y, rv.pdf(pos))

In [ ]:
rv = scipy.stats.multivariate_normal([0.5, -0.2])
plt.contourf(x, y, rv.pdf(pos))

We can take a sample from this distribution we just created.

In [ ]:
rv.rvs(500)

## Step 2: Write some univariate kernel
Let's use the Gaussian kernel first.

In [ ]:
from functools import partial

In [ ]:
def gaussian_univariate_kernel(b, x):
    '''
    x is the distance
    b is the bandwidth
    '''
    c = (2*np.pi)**(-0.5)
    return c * np.exp(-0.5*(x/b)**2)

In [ ]:
Kh = partial(gaussian_univariate_kernel, 1)

In [ ]:
xs = np.linspace(-5, 5, 100)
ys = Kh(xs)
px.scatter( x = xs, y = ys)

## Step 3: Write a product kernel in two dimensions
A product kernel is not the same thing as a diagonal bandwidth matrix

In [ ]:
def product_kernel(h, x):
    '''
    h is a d-dimensional vector of smoothing parameters for each dimension
    x is a single d-dimensional vector representing the distance
    '''
    c = 1/np.prod(h)
    v = gaussian_univariate_kernel(h, x)
    return c * np.prod(v, axis=-1)

*Testing for the following properties:*
- The product kernel takes the product of the univariate kernel in each dimension
- The product kernel handles vectors properly


In [ ]:
product_kernel(np.array([1, 1]), np.array([0, 0])) == \
    gaussian_univariate_kernel(1, 0) * gaussian_univariate_kernel(1, 0)

In [ ]:
product_kernel(np.array([1, 2]), np.array([0, 1])) == \
    1/2*gaussian_univariate_kernel(1, 0) * gaussian_univariate_kernel(2, 1)

In [ ]:
np.array([product_kernel(np.array([1, 2]), np.array([0, 1])), product_kernel(np.array([1, 2]), np.array([1, 1]))]) == \
    product_kernel(np.array([1, 2]), np.array([[0, 1],[1,1]]))

## Step 4: Test out product kernel
We first simulate one Gaussian distribution in two dimensions centered at (0.5, -0.2).

In [ ]:
rv = scipy.stats.multivariate_normal([0.5, -0.2])
xs = rv.rvs(2000)

Evaluate sum of kernels at origin.

In [ ]:
smoothings = np.array([1, 1])

In [ ]:
def product_kernel_estimation(h, xs, x):
    return np.mean(product_kernel(h, xs-x))

In [ ]:
# our point is (0, 0) so each xs serves as the difference vector
product_kernel_estimation(smoothings, xs, np.array([0, 0])) == \
    np.mean(product_kernel(smoothings, xs))

Plot this stuff.

In [ ]:
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)

In [ ]:
K = partial(product_kernel_estimation, smoothings, xs)
px.scatter(x=xs[:, 0], y=xs[:, 1], color=[K(x) for x in xs])

Wow. That looks convincing that it worked. Next is trying it in more than two dimensions.

## Step 5: Compare it to scipy's gaussian_kde
That seems like it worked. Relevant would be the scalability of the algorithm? Or do we not care about performance yet?

In [ ]:
scipy_kde = scipy.stats.gaussian_kde(xs.T)
px.scatter(x=xs[:, 0], y=xs[:, 1], color=scipy_kde(xs.T))